# isee_machine & isee_quality
- isee_quality의 모든 정보가 'NG' 즉, 모두 비정상이라는 가정하에 매칭해봄
- NG 라벨링 part
- 18.06.12

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
isee_machine = pd.read_csv('./data/isee_machine_data_20180901_to_20190331_C123.csv',encoding= 'cp949')

# 시간 type 변경
isee_machine['creationTime'] = pd.to_datetime(isee_machine['creationTime'],format = '%m/%d/%Y %H:%M')

# 인덱스 오름차순으로 재정렬
isee_machine = isee_machine.sort_index(ascending = False).reset_index(drop=True)

In [3]:
# 시간차 계산 (초)
creationtime_diff = []
creationtime = isee_machine['creationTime']
for i in range(creationtime.shape[0]):
    if i == 0:
        creationtime_diff.append(0)
    else:
        diff_seconds = (creationtime[i] - creationtime[i-1]).total_seconds()
        creationtime_diff.append(diff_seconds)
isee_machine['creationTime_diff'] = creationtime_diff   

In [4]:
print('총 데이터 건: ',isee_machine.shape[0])

총 데이터 건:  236696


### cycle 컬럼 추가
- cycle 기준(초) > 1h == 3600 sec

In [5]:
cycle_std_sec = 3600

last_cycle_idx = isee_machine[isee_machine.creationTime_diff >= cycle_std_sec].index.tolist()
last_cycle_idx.append(len(isee_machine)-1)  #-- 마지막 index 추가

cycle_num = []
for i, last_idx in enumerate(last_cycle_idx):
    if i == 0:
        frist_idx = 0
    elif i == len(last_cycle_idx)-1:
        last_idx = last_idx+1
        
    num = [i+1 for _ in range(last_idx-frist_idx)]
    cycle_num.extend(num)
        
    frist_idx = last_idx

isee_machine['CycleNum'] = cycle_num
cycle_len = len(set(cycle_num))

In [6]:
# cycle 첫 시작 creationTime_difff를 0으로 
for i in range(1,cycle_len+1):
    #-- 마지막 cycle의 마지막 데이터 creationTime_diff = 3600 > 60으로
    idx = isee_machine[isee_machine.CycleNum == i].index[0]
    isee_machine.loc[idx,'creationTime_diff'] = 0
        

#### 동일 시간은 평균냄

In [7]:
use_cols = ['ModuleCount', 'CycleTime', 'VpPosition', 'VpTime','VpPressure', 'MaxInjectionPressure', 'InjectionEndPosition',
       'PlasticizingTime', 'PlasticsEndPosotion', 'ClampingTime','MouldOpeningTime', 'HotOilContoller', 'TankTemp', 'NozzleTemp',
       'Zone1', 'Zone2', 'Zone3', 'Zone4','CycleNum']

data1 = isee_machine.groupby(['creationTime'])[use_cols].mean()

data2 = isee_machine.groupby(['creationTime'])[['creationTime_diff']].sum()
new_data = pd.merge(data1,data2, how = 'inner',left_index=True, right_index=True)

new_data = new_data.reset_index()
new_data.shape

(142480, 21)

#### 보간:

In [8]:
isee_machine_inter = pd.DataFrame(columns = use_cols)
for num in new_data['CycleNum'].unique().tolist():
# num = 1
    all_time_data = new_data[new_data.CycleNum == num].set_index('creationTime')[use_cols].resample('1T').interpolate().resample('1T').asfreq().dropna()
    all_time_data['CycleNum'] = [np.int(num) for _ in range(len(all_time_data))]

    isee_machine_inter = pd.concat([isee_machine_inter,all_time_data], axis = 0)
    
isee_machine_inter = isee_machine_inter.reset_index().rename(columns = {'index':'creationTime'}) 
isee_machine_inter.shape

(147890, 20)

In [9]:
# 일자컬럼 추가 (object)
isee_machine_inter['creationDate'] = isee_machine_inter['creationTime'].dt.date.astype(str)

In [10]:
isee_machine_inter['creationHour'] = isee_machine_inter['creationTime'].dt.hour

In [11]:
isee_machine_inter['YM'] = isee_machine_inter['creationTime'].dt.strftime('%Y-%m').astype(str)

### 가동 (초기,중기, 말기) 라벨링
- beginning_std_min : 초기기준 (시각~ 00분까지 / 분으로 계산하여 삽입)
- end_std_min: 말기기준(00분 ~ 끝 /  분으로 계산하여 삽입)

In [12]:
beginning_std_min = 5
end_std_min = 5

operation_label = []
cycle_num = list(isee_machine_inter['CycleNum'].unique())
for c in cycle_num : 
    data = isee_machine_inter[isee_machine_inter.CycleNum == c ].reset_index(drop = True)
    frist_cycle_time = data['creationTime'].iloc[0]
    last_cycle_time = data['creationTime'].iloc[-1]
    cnt = 0
    for idx, time in enumerate(data['creationTime']):
        if time <= (frist_cycle_time + timedelta(minutes = +beginning_std_min)):  # 초기 
            operation_label.append(0)
        elif time >= (last_cycle_time + timedelta(minutes = -end_std_min)): # 말기 
            operation_label.append(2)
        else : 
            operation_label.append(1)
        
isee_machine_inter['Operation_label'] = operation_label

In [13]:
### 실제 라벨링# 한 cycle에 다른 날짜 확인 >  63개
oneCycle_diffDate = {}
cycle_num = list(isee_machine_inter['CycleNum'].unique())
for c in cycle_num : 
    data = isee_machine_inter[isee_machine_inter.CycleNum == c ].reset_index(drop = True)
    date = data['creationDate'].unique()
    if date.shape[0] != 1:
        oneCycle_diffDate[c] = date
len(oneCycle_diffDate)

63

### NG라벨링
- quality 파일에 있는 모든 데이터를 이상이라고 봄

In [14]:
quality = pd.read_csv('./data/isee_quality_data_20180901_to_201902.csv',encoding= 'cp949')
quality['CreationTime']  = pd.to_datetime(quality['CreationTime'],format = '%m/%d/%Y %H:%M')

qua_createtime = pd.Series(quality.groupby(['CreationTime']).count().reset_index()['CreationTime'])

- Qua_Mach_YN(quality와 매칭되지 않는 기간) == 0

In [15]:
isee_machine_inter['Qua_Match_YN'] = 0
isee_machine_inter['Qua_Match_YN'].loc[isee_machine_inter['creationTime'] <= qua_createtime.iloc[-1]] = 1

In [16]:
mer_df = pd.merge(isee_machine_inter[isee_machine_inter.Qua_Match_YN == 1], qua_createtime, how = 'left',left_on='creationTime', right_on='CreationTime')

mer_df['CreationTime'].loc[~mer_df.CreationTime.isnull()] = 1
mer_df['CreationTime'].loc[mer_df.CreationTime.isnull()] = 0

In [17]:
mer_df = mer_df.rename(columns = {'CreationTime':'NG'})

In [18]:
print('매칭되지 않은 라벨 건: ',len(qua_createtime) - len(mer_df[mer_df.NG == 1]['creationTime'].unique()))
print('매칭된 라벨 건: ',len(mer_df[mer_df.NG == 1]['creationTime'].unique()))

매칭되지 않은 라벨 건:  340
매칭된 라벨 건:  4879


### NG5 라벨링
- 불량 시간 5분전도 불량으로 라벨링

In [19]:
# 불량 발생 5분 전도 불량으로 책정
add_f_min = 5 
add_date = []
for date in qua_createtime:
    minutes = 0
    for i in range(1,add_f_min+1):   
        minutes = i*60
        add_date.append(date-timedelta(0,minutes))
add_date = pd.Series(add_date)        

In [20]:
qua_createtime_5 = pd.Series(pd.concat([qua_createtime,add_date]).unique(),name = 'NG5')

In [21]:
mer_df = pd.merge(mer_df, qua_createtime_5, how = 'left',left_on='creationTime', right_on='NG5')

mer_df['NG5'].loc[~mer_df.NG5.isnull()] = 1
mer_df['NG5'].loc[mer_df.NG5.isnull()] = 0

In [22]:
# mer_df[mer_df.creationTime.astype(str).str.startswith('2018-09-07 02:2')][['creationTime','NG','NG5']]

In [23]:
print('qua_createtime_5 : ',len(qua_createtime_5))
print('매칭되지 않은 라벨 건: ',len(qua_createtime_5) - len(mer_df[mer_df.NG5 == 1]['creationTime'].unique()))
print('매칭된 않은 라벨 건: ',len(mer_df[mer_df.NG5 == 1]['creationTime'].unique()))

qua_createtime_5 :  18883
매칭되지 않은 라벨 건:  1179
매칭된 않은 라벨 건:  17704


In [24]:
print('총 데이터 건: ',mer_df.shape[0])
print('cycle 수: ',mer_df['CycleNum'].unique().shape[0])
print('NG라벨 불량 건: ',mer_df[mer_df.NG == 1].shape[0])
print('NG라벨 정상 건: ',mer_df[mer_df.NG != 1].shape[0])
print('NG5라벨 불량 건: ',mer_df[mer_df.NG5 == 1].shape[0])
print('NG5라벨 정상 건: ',mer_df[mer_df.NG5 != 1].shape[0])

총 데이터 건:  118793
cycle 수:  124
NG라벨 불량 건:  4879
NG라벨 정상 건:  113914
NG5라벨 불량 건:  17704
NG5라벨 정상 건:  101089


In [25]:
mer_df.to_csv('./data/machine_NG_inter.csv',index = False)

### 원본(isee_machine)에 NG, NG5 붙이기
- NG : 0정상, 1이상
- NG5 : 0정상, 1이상
##### ***관련 없는 (Qua_Match_YN == 1) 부분도 정상으로 됨

In [26]:
isee_machine_inter['NG'] = 0
isee_machine_inter['NG'].iloc[mer_df.index] = mer_df['NG']

In [27]:
isee_machine_inter['NG5'] = 0
isee_machine_inter['NG5'].loc[mer_df.index] = mer_df['NG5']

In [28]:
print('총 데이터 건: ',isee_machine_inter.shape[0])
print('cycle 수: ',isee_machine_inter['CycleNum'].unique().shape[0])
print('NG라벨 불량 건: ',isee_machine_inter[isee_machine_inter.NG == 1].shape[0])
print('NG라벨 정상 건: ',isee_machine_inter[isee_machine_inter.NG == 0].shape[0])
print('NG5라벨 불량 건: ',isee_machine_inter[isee_machine_inter.NG5 == 1].shape[0])
print('NG5라벨 정상 건: ',isee_machine_inter[isee_machine_inter.NG5 == 0].shape[0])

총 데이터 건:  147890
cycle 수:  160
NG라벨 불량 건:  4879
NG라벨 정상 건:  143011
NG5라벨 불량 건:  17704
NG5라벨 정상 건:  130186


In [29]:
isee_machine_inter.to_csv('./data/isee_machine_NG_inter.csv',index = False)

### 불량 cycle check

In [30]:
# y_cycle_num = list(mer_df[mer_df.NG == 1]['CycleNum'].unique())

# for num in y_cycle_num:
#     date = list(mer_df[(mer_df.NG == 1) & (mer_df.CycleNum == num)]['creationDate'].unique())
#     print('Cycle {} : {}'.format(num,date))

In [31]:
# cycle별 시작 시간과 끝난 시간

# isee_machine_inter = pd.read_csv('./data/isee_machine_NG.csv')
# isee_machine_inter['creationTime'] = pd.to_datetime(isee_machine_inter['creationTime'])

fl_dic = {}
for i in np.unique(isee_machine_inter['CycleNum']):
    df = isee_machine_inter[isee_machine_inter['CycleNum']==i]
    df = df.reset_index(drop=True)
    firsttime = df.loc[0]['creationTime']
    lasttime = df.loc[len(df)-1]['creationTime']
    fl_dic[i] = (firsttime, lasttime)

time_df = pd.DataFrame(fl_dic).T
time_df = time_df.reset_index()
time_df.columns = ['CycleNum','start','finish']

time_df['s_date'] = time_df.start.dt.strftime('%Y-%m-%d').astype(str)
time_df['s_year'] = time_df.start.dt.strftime('%Y').astype(str)
time_df['s_month'] = time_df.start.dt.strftime('%m').astype(str)
time_df['s_day'] = time_df.start.dt.strftime('%d').astype(str)
time_df['s_hour'] = time_df.start.dt.strftime('%H').astype(str)
time_df['s_miniute'] = time_df.start.dt.strftime('%M').astype(str)

time_df['f_date'] = time_df.finish.dt.strftime('%Y-%m-%d').astype(str)
time_df['f_year'] = time_df.finish.dt.strftime('%Y').astype(str)
time_df['f_month'] = time_df.finish.dt.strftime('%m').astype(str)
time_df['f_day'] = time_df.finish.dt.strftime('%d').astype(str)
time_df['f_hour'] = time_df.finish.dt.strftime('%H').astype(str)
time_df['f_miniute'] = time_df.finish.dt.strftime('%M').astype(str)
time_df['time_diff_sec'] =(time_df.finish - time_df.start).dt.total_seconds().astype(int)

time_df.drop(['start','finish'],axis=1).to_csv('./result/isee_machine/cycle_start_end_inter.csv',
                                              index = False)